## 1.Check how many routing paths exist in the data set.

In [19]:
import pandas as pd
import os
import networkx as nx
import numpy as np

In [54]:
def check_path(graph, sample_num = 10):
    ## Find the end-station
    degree_dict = dict(graph.degree())
    degree_one_nodes = [node for node, degree in degree_dict.items() if degree == 1]

    # Randomly select 10 pairs of talker and listeners
    talkers = np.random.choice(degree_one_nodes, sample_num)
    listener = np.random.choice(degree_one_nodes, sample_num)

    # Replace the listeners same as talkers
    while np.any(talkers == listener):
        listener = np.where(talkers == listener, np.random.choice(degree_one_nodes, 10), listener)

    number_of_paths = []
    for i in range(sample_num):
        source_node = talkers[i]
        target_node = listener[i]
        all_paths = nx.all_simple_paths(graph, source=source_node, target=target_node)
        number_of_paths.append(sum(1 for _ in all_paths))
    return number_of_paths

In [55]:
def check_path_num(topo_type, sample_num = 10):
    net_list = []
    for path in [x for x in os.listdir(topo_type) if 'topo' in x]:
        topo = pd.read_csv(f'{topo_type}/' + path)
        topo['source'] = topo['link'].apply(lambda x: eval(x)[0])
        topo['target'] = topo['link'].apply(lambda x: eval(x)[1])
        net_list.append(topo)

    for conf in net_list:
        # Create a graph from the dataframe
        graph = nx.from_pandas_edgelist(conf, source='source', target='target')
        assert isinstance(graph, nx.classes.graph.Graph) == True
        result = check_path(graph, sample_num)
        if topo_type in ['line', 'tree']:
            if not all(x == 1 for x in result ):
                print(f'Error in {topo_type} topology', result )
        elif topo_type == 'ring':
            if not all(x == 2 for x in result ):
                print(f'Error in {topo_type} topology', result )
        else:
            if not all(x > 1 for x in result ):
                print(f'Error in {topo_type} topology', result )

In [56]:
check_path_num('line')

In [57]:
check_path_num('tree')

In [58]:
check_path_num('ring')

In [59]:
check_path_num('mesh')